<a href="https://colab.research.google.com/github/Alsr96/LMMAES/blob/master/LMMAES_evo_paths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import math
from keras import backend as K

class structtype():
    pass

def nw_to_vec(model,layer_idx=None):
    n_layers = len(model.layers)
    vector=np.empty((0,))
    ind=np.zeros((1,))
    sum_i=0
    if layer_idx==None:
        idx=range(n_layers)
    else:
        idx=layer_idx
    for i in idx:
        if len(model.layers[i].get_weights())==2:
            weights, biases = model.layers[i].get_weights()
            s_w=np.size(weights)
            sum_i=sum_i+s_w
            ind=np.append(ind,sum_i)
            w_v=np.reshape(weights,(s_w,))
            s_b=np.size(biases)
            sum_i=sum_i+s_b
            ind=np.append(ind,sum_i)
            b_v=np.reshape(biases,(s_b,))
            wb=np.append(w_v,b_v)
            vector=np.append(vector,wb)
    return vector, ind

def vec_to_nw(vector,ind,model,layer_idx=None):
    n_layers = len(model.layers)
    if layer_idx==None:
        idx=range(n_layers)
    else:
        idx=layer_idx
    k=0
    for i in idx:
        if len(model.layers[i].get_weights())==2:
            weights,biases=model.layers[i].get_weights()
            j1=k
            j2=k+1
            j3=k+2
            weights=np.reshape(vector[int(ind[j1]):int(ind[j2])],np.shape(weights))
            biases=np.reshape(vector[int(ind[j2]):int(ind[j3])],np.shape(biases))
            model.layers[i].set_weights((weights,biases))
            k=k+2
    return model

#split=[15,15]
def split_data(input_data,target2,split):
    n_samples=len(target2)
    s1_input=input_data[0:int(split[0]*n_samples/(split[0]+split[1]))]
    s1_target=target2[0:int(split[0]*n_samples/(split[0]+split[1]))]
    s2_input=input_data[int(split[0]*n_samples/(split[0]+split[1])):n_samples]
    s2_target=target2[int(split[0]*n_samples/(split[0]+split[1])):n_samples]
    return s1_input, s1_target, s2_input, s2_target

# obtain gradients
def get_gradients(model,inputv,output):
  grads = model.optimizer.get_gradients(model.total_loss, model.trainable_weights)
  symb_inputs = (model._feed_inputs + model._feed_targets + model._feed_sample_weights)
  f = K.function(symb_inputs, grads)
  x, y, sample_weight = model._standardize_user_data(inputv, output)
  output_grad = f(x + y + sample_weight)
  return output_grad

class lmmaes(object):
    def __init__(self,model,n_candidates = None,mu = None, m = None,sigma=1/10,function_budget=10000,function_target=None,layer_idx=None):
      self.sigma=sigma
      # converting the weights and biases to a row vector
      self.layer_idx=layer_idx
      self.y, self.ind = nw_to_vec(model,layer_idx=self.layer_idx)
      
      # number of layers to optimise
      if self.layer_idx==None:
          self.n_layers = len(model.layers)
          itr = range(self.n_layers)
      else:
          self.n_layers = len(self.layer_idx)
          itr = self.layer_idx

      # calculating number of dimensions
      self.n_dimensions=0
      for i in itr:
          if len(model.layers[i].get_weights())==2:
              weights, biases = model.layers[i].get_weights()
              self.n_dimensions=np.size(weights)+np.size(biases)+self.n_dimensions

      # number of candidate solutions generated
      self.n_candidates=n_candidates
      if self.n_candidates==None:
          self.n_candidates = 2*int((4 + np.floor(3*np.log(self.n_dimensions)))/2)

      if mu==None:
        # number of best solutions selected
        self.mu = int(self.n_candidates/2)
      else:
        self.mu=mu

      # weights for selected solutions
      self.w = np.empty([0,0])
      for i in range(int(self.mu)):
          self.w = np.append(self.w,np.log(self.mu+0.5)-np.log(i+1))
      sum_w = np.sum(self.w)
      self.w = self.w/sum_w


      self.mu_w = 1/(np.sum(np.square(self.w)))

      if m==None:
        # number of evolution paths
        self.m = self.n_candidates
      else:
        self.m=m

      self.c_sigma = 2*self.n_candidates/self.n_dimensions
      self.const1=np.sqrt(self.mu_w*self.c_sigma*(2-self.c_sigma))

      # learning rates
      self.c_d = np.empty([0,0])
      self.c_c = np.empty([0,0])
      self.const2=np.empty([0,0])
      for i in range(int(self.m)):
          self.c_d = np.append(self.c_d,1/(self.n_dimensions*(1.5**i)))
          self.c_c = np.append(self.c_c,self.n_candidates/(self.n_dimensions*(4**i)))
          self.const2=np.append(self.const2,np.sqrt(self.mu_w*self.c_c[i]*(2-self.c_c[i])))
      #print(self.c_c,self.c_d,self.const1,self.const2)
      self.t=0

      # length of evolution paths (exponentially fading record of recent most successful steps)
      self.p_sigma = np.zeros((self.n_dimensions,))

      # vectors modelling deviation of transformation matrix from identity matrix
      self.m_i = np.zeros((int(self.m), self.n_dimensions))
    
    # 1 step of optimisation
    def train_on_batch(self,model,train_data,divide_data=False,use_gradients=False):
      self.use_gradients=use_gradients
      self.func_calls=0
      train_input=train_data[0]
      train_target=train_data[1]
      # sampling a normal distribution
      samples=np.random.randn(int(self.n_candidates/2),self.n_dimensions)
      z=np.zeros((self.n_candidates,self.n_dimensions))
      for i in range(int(self.n_candidates/2)):
        z[2*i]=samples[i]
        z[2*i+1]=-1*samples[i]
      d = np.copy(z)
      #print(np.sum(d))
      f_list=np.empty((int(self.n_candidates),1))

      for j in range(min(self.t, self.m)):
            d = ((1 - self.c_d[j])*d) + (self.c_d[j] *np.outer(np.dot(d, self.m_i[j, :]), self.m_i[j, :]))
      
      for i in range(int(self.n_candidates)):
          model=vec_to_nw(self.y+self.sigma*d[i],self.ind,model,layer_idx=self.layer_idx)
          # evaluating the loss
          num_samples=len(train_target)-1
          if divide_data==False:
            res=model.evaluate(x=train_input,y=train_target,verbose=0)
          else:
            res=model.evaluate(x=train_input[int(i*num_samples/self.n_candidates):int((i+1)*num_samples/self.n_candidates)],y=train_target[int(i*num_samples/self.n_candidates):int((i+1)*num_samples/self.n_candidates)],verbose=0)
          self.func_calls=self.func_calls+1
          f_list[i][0] = res[0]
      

      # sorting the solutions based on fitness
      sortidx_f = f_list.argsort(axis=0)
      # selecting the best 'mu' out of the 'lambda' mutations
      sortidx_f = sortidx_f[0:int(self.mu)]
      best_list = np.empty([int(self.mu),self.n_dimensions])
      j = 0
      # weighted average of the best mutations
      for i in sortidx_f:
          best_list[j] = self.w[j]*d[i]
          j = j+1
      
      y_update = self.sigma*np.sum(best_list,0)
      y_next = self.y + y_update

      self.y = y_next
      model=vec_to_nw(self.y,self.ind,model,layer_idx=self.layer_idx)
      
      best_list2 = np.empty([int(self.mu),self.n_dimensions])
      j = 0
      for i in sortidx_f:
          best_list2[j] = self.w[j]*z[i]
          j = j+1
      weighted_sum_norm=np.sum(best_list2,0)
      p_sigma_next = (1-self.c_sigma)*self.p_sigma + self.const1*weighted_sum_norm
      mag_p_sigma_next = np.linalg.norm(p_sigma_next)
      
      m_update=weighted_sum_norm

      if self.use_gradients==True:
        print('lololol')
        gradients=get_gradients(model,train_input,train_target)
        gradient_vector=np.empty((0,0))
        for i in range(len(gradients)):
          gradient_vector=np.append(gradient_vector,gradients[i].reshape(np.size(gradients[i]),))
        gradient_vector=np.linalg.norm(weighted_sum_norm)*(-1*gradient_vector/np.linalg.norm(gradient_vector))
        m_update=gradient_vector

      # M update
      for i in range(int(self.m)):
          self.m_i[i] = (1-self.c_c[i])*self.m_i[i] + self.const2[i]*m_update
      sigma_next = self.sigma*np.exp(self.c_sigma*(((mag_p_sigma_next**2)/self.n_dimensions)-1)/2)
      self.t=self.t+1
      self.sigma = sigma_next
      self.p_sigma = p_sigma_next
      jhlk=sortidx_f[0][0]
      #print(f_list[int(jhlk)],self.t,self.sigma)
      return model

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

from keras.datasets import mnist
#download mnist data and split into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train=X_train.reshape(60000,28,28,1)
X_test=X_test.reshape(10000,28,28,1)

from keras.utils import to_categorical
#one-hot encode target column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [0]:
#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

xtrain,ytrain,xval,yval=split_data(X_train,y_train,[5,1])

In [0]:
import numpy as np
lm=lmmaes(model,n_candidates = 40, mu=15, m=40, sigma=1/100,layer_idx=None)

batch=int(1000)
no_of_epochs=5
print('Number of dimensions: %g; lambda: %g; mu: %g; m: %g; batch size: %g, no. of epochs: %g'%(lm.n_dimensions,lm.n_candidates,lm.mu,lm.m,batch,no_of_epochs))
run_settings=lm.n_dimensions,lm.n_candidates,lm.mu,lm.m,lm.sigma,batch,no_of_epochs
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

res_current_best=model.evaluate(x=xval,y=yval)
y_current_best,ind=nw_to_vec(model,layer_idx=None)
validation_fail=0
log_train=np.array([-1,0,lm.sigma,res_current_best[0],res_current_best[1]])
log_validation=np.array([-1,res_current_best[0],100*res_current_best[1]])

In [0]:
for i in range(no_of_epochs):
  # current best solution
  #xtrain,ytrain=unison_shuffled_copies(xtrain,ytrain)
  #print(batch/lm.n_candidates)
  no_samples=len(ytrain)-1
  n_groups=int(no_samples/batch)
  
  for j in range(n_groups):
    # 1 step of optimisation
    model = lm.train_on_batch(model,train_data=(xtrain[int(j*no_samples/n_groups):int((j+1)*no_samples/n_groups)],ytrain[int(j*no_samples/n_groups):int((j+1)*no_samples/n_groups)]),divide_data=False)
    res_new=model.evaluate(x=xtrain[int(j*no_samples/n_groups):int((j+1)*no_samples/n_groups)],y=ytrain[int(j*no_samples/n_groups):int((j+1)*no_samples/n_groups)],verbose=0)
    log_train=np.append(log_train,[i,j,lm.sigma,res_new[0],res_new[1]],axis=0)
    print('epoch: %g;batch: %g; sigma: %g; loss: %.2f; accuracy: %.2f' % (i,j,lm.sigma,res_new[0],100*res_new[1]))
  # validation
  res_val=model.evaluate(x=xval,y=yval)
  log_validation=np.append(log_validation,[i,res_val[0],100*res_val[1]],axis=0)
  if res_new[1]>res_current_best[1]:
    res_current_best=res_new
    #y_current_best=y_new
    validation_fail=0
    y_current_best,ind=nw_to_vec(model)
  else:
    #model=vec_to_nw(y_current_best,ind,model,layer_idx=None)
    validation_fail=validation_fail+1
  
  if validation_fail>=6:
    break

In [0]:
model=vec_to_nw(y_current_best,ind,model)
test_result=model.evaluate(x=X_test,y=y_test)
print(test_result[1])

In [0]:
execution_time=1722.666
file_name='nonevo8.npz'
np.savez(file_name,run_settings=run_settings,log_train=log_train,log_validation=log_validation,test_result=test_result,evo_paths=True)

In [0]:
from google.colab import files
files.download(file_name)

In [0]:
import matplotlib.pyplot as plt
count_less=0
count_more=0
for i in range(int(len(log_diff)/3)):
  if i!=0:
    if log_diff[3*i+2]<90:
      count_less=count_less+1
      #plt.scatter(i,-1)
    else:
      count_more=count_more+1
      #plt.scatter(i,1)
    plt.scatter(i,count_more/count_less)
plt.show()
print(count_less,count_more)

In [0]:
import matplotlib.pyplot as plt
count_less=0
count_more=0
f1=plt.figure(1)
f2=plt.figure(2)
for i in range(int(len(log_diff)/3)):
  if i!=0:
    if log_diff[3*i+2]<90:
      count_less=count_less+1
      #plt.scatter(i,-1)
    else:
      count_more=count_more+1
      #plt.scatter(i,1)
    f1
    plt.scatter(i,count_more,)
    f2
    plt.scatter(i,count_less)
f1
plt.show()
f2
plt.show()
print(count_less,count_more)

In [0]:
import matplotlib.pyplot as plt
for i in range(int(len(log_train)/5)):
  plt.scatter(i*10,1000*log_train[i*5+2])
plt.show()

In [0]:
import matplotlib.pyplot as plt
for i in range(int(len(log_train)/5)):
  plt.scatter(i,log_train[i*5+4])
plt.show()

In [0]:
import matplotlib.pyplot as plt
for i in range(int(len(log_validation)/3)):
  plt.scatter(i,log_validation[3*i+2])
plt.show()

In [0]:
log_diff2=np.empty((0,0))
for i in range(int(len(log_diff)/3)):
  if i!=0:
    log_diff2=np.append(log_diff2,log_diff[i*3+2])
print(np.amax(log_diff2))
print(np.amin(log_diff2))